In [1]:
import os
from collections import defaultdict, Counter
from text_utils import remove_reg, tokenise, get_provenance
import json
import numpy as np
from tqdm.notebook import tqdm
from wiki_labels import qid_lab_get

BASE_DIR, DB_DIR = '/projekti/mondoAPI', 'db'

types = json.load(open(os.path.join(BASE_DIR, DB_DIR, 'types.json'), 'r'))
types_a = np.array([tuple([b if b else '' for b in a]) if len(a) == 4 else ('', '', '', '') for a in types],
                   [('tip', 'U3'), ('lang', 'U2'), ('script', 'U4'), ('cc', 'U2')])
t2i = {'_'.join(v):i for i, v in enumerate(types_a)}
print(qid_lab_get(177053, include_alt=True)['mr'])
print(qid_lab_get(191701, include_alt=True)['mba'])

{'en'}
{'uk', 'en', 'nb', 'lb', 'lt', 'nn', 'ru', 'cs', 'fi', 'da', 'vi', 'zh', 'ar', 'fr', 'hu', 'pt', 'kk', 'my', 'fa', 'id', 'sv', 'tr', 'eu'}


In [39]:
i = 0
names = defaultdict(Counter)
nema = Counter()
for l in tqdm(open('/backup/wikidata/fnln.txt'), total=26_332_292):
    try:
        qid, n, lang, fn, ln, tip, gender = l.strip('\n').split('\t')
    except Exception as e:
        print(e, l)
    if fn and ln:
        provs = set([a[:-2] for a in get_provenance(n, lang)])
        for prov in provs:
            tag_t = 'fn' + ('1' if gender == 'm' else ('2' if gender == 'f' else '0'))
            tag = tag_t +'_'+prov
            names[fn]['PER_'+tag] += 1
            tag = 'ln' + '_' + prov
            names[ln]['PER_'+tag] += 1
names['prof.'], names['doctor'], names['davor'], names['johnson']

  0%|          | 0/26332292 [00:00<?, ?it/s]

(Counter(),
 Counter({'PER_ln_nn_Latn_': 1,
          'PER_ln_ca_Latn_': 12,
          'PER_ln_sl_Latn_': 4,
          'PER_ln_sq_Latn_': 4,
          'PER_ln_nl_Latn_': 16,
          'PER_ln_es_Latn_': 14,
          'PER_ln_it_Latn_': 1,
          'PER_ln_sv_Latn_': 1,
          'PER_ln_en_Latn_': 16,
          'PER_ln_da_Latn_': 1,
          'PER_ln_nb_Latn_': 1,
          'PER_ln_fr_Latn_': 4,
          'PER_ln_ga_Latn_': 2,
          'PER_ln_de_Latn_': 14,
          'PER_ln_cs_Latn_': 12,
          'PER_ln_pt_Latn_': 2,
          'PER_ln_eo_Latn_': 1,
          'PER_ln_sk_Latn_': 1}),
 Counter({'PER_fn1_pt_Latn_': 12,
          'PER_fn1_ca_Latn_': 7,
          'PER_fn1_sl_Latn_': 9,
          'PER_fn1_fi_Latn_': 6,
          'PER_fn1_nb_Latn_': 6,
          'PER_fn1_sq_Latn_': 6,
          'PER_fn1_nl_Latn_': 13,
          'PER_fn1_es_Latn_': 10,
          'PER_fn1_it_Latn_': 6,
          'PER_fn1_pl_Latn_': 6,
          'PER_fn1_da_Latn_': 6,
          'PER_fn1_en_Latn_': 20,
    

In [41]:
TODO = defaultdict(set)
for l in tqdm(open('data/wikiid.tsv'), total=834281):
    l = l.strip('\n')
    tip, qid = l.strip('\n').split('\t')
    if qid[0] == 'Q':
        if tip in ('TITLE', 'RANK', 'POS',):
            tip = 'tit'
        else:
            tip = tip.lower()

        qid = int(qid[1:])
        TODO[qid].add(tip)
TODO[121594], TODO[177053], len(TODO)

  0%|          | 0/834281 [00:00<?, ?it/s]

({'tit'}, {'tit'}, 790368)

In [42]:
freq_nelma = Counter()
for l in tqdm(open('/backup/wikidata/wiki_nelma.jsonl'), total=10558506):
    j = json.loads(l)
    if 'title' in j:
        for qtit in j['title']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1
            if qtit not in TODO:
                TODO[qtit].add('tit')
            elif 'tit' not in TODO[qtit]:
                TODO[qtit].add('tit')
    if 'fn' in j:
        for qtit in j['fn']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1
    if 'ln' in j:
        for qtit in j['ln']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1


  0%|          | 0/10558506 [00:00<?, ?it/s]

In [43]:
for qid, tips in tqdm(TODO.items(), total=len(TODO)):
    lab = qid_lab_get(qid, include_alt=True)
    for l, langs in lab.items():
        for lang in langs:
            for tip in tips:
                provs = set([a[:-2] for a in get_provenance(l, lang)])
                for prov in provs:
                    tag = f'{tip}_'+prov
                    #print(tag, tag in t2i)
                    if tag in t2i:
                        names[l][f'W_{tag}'] = freq_nelma[qid]
                    else:
                        names[l][f'W_{tip}__unkn_'] = freq_nelma[qid]
                        nema[tag] += 1

names['prof'], names['mba']

  0%|          | 0/798119 [00:00<?, ?it/s]

(Counter({'W_tit_sl_Latn_': 4043,
          'W_tit_de_Latn_': 4043,
          'W_tit_wa_Latn_': 120,
          'W_tit_fr_Latn_': 10}),
 Counter({'W_ln__unkn_': 1,
          'W_fn0__unkn_': 1,
          'W_tit_uk_Latn_': 222,
          'W_tit_en_Latn_': 222,
          'W_tit_nb_Latn_': 222,
          'W_tit_lb_Latn_': 222,
          'W_tit_lt_Latn_': 222,
          'W_tit_nn_Latn_': 222,
          'W_tit_ru_Latn_': 222,
          'W_tit_cs_Latn_': 222,
          'W_tit_fi_Latn_': 222,
          'W_tit_da_Latn_': 222,
          'W_tit_vi_Latn_': 222,
          'W_tit_zh_Latn_': 222,
          'W_tit_ar_Latn_': 222,
          'W_tit_fr_Latn_': 222,
          'W_tit_hu_Latn_': 222,
          'W_tit_pt_Latn_': 222,
          'W_tit_kk_Latn_': 222,
          'W_tit_my_Latn_': 222,
          'W_tit_fa_Latn_': 531,
          'W_tit_id_Latn_': 222,
          'W_tit_sv_Latn_': 222,
          'W_tit_tr_Latn_': 222,
          'W_tit_eu_Latn_': 222,
          'W_tit_he_Latn_': 786}))

In [77]:
toks = defaultdict(Counter)
for tok, v in tqdm(names.items(), total=len(names)):
    for t_prov, cnt in v.items():
        src, t, lang, script, cc = t_prov.split('_')
        toks[f'{tok}_{lang}_{script}'][(src, t)] += cnt
        if tok[-1] == '.' and tok.count('.') == 1:
            toks[f'{tok[:-1]}_{lang}_{script}'][(src+'.', t)] += cnt
toks = dict(toks)
toks['dr_en_Latn'], toks['mba_en_Latn']

  0%|          | 0/1820542 [00:00<?, ?it/s]

(Counter({('W.', 'tit'): 1486}), Counter({('W', 'tit'): 222}))

In [93]:
t2set = defaultdict(set)
for t, c in t2i.items():
    t2 = t.split('_')
    t2 = '_'.join(t2[:3])
    t2set[t2].add(c)
t2set = dict(t2set)    
i2t = {k: v for v, k in t2i.items()}
t2set['tit_hr_Latn'], i2t[1074]

({1074}, 'tit_hr_Latn_')

In [121]:
freqs = {}
nema = Counter()
for k, v in tqdm(toks.items(), total=len(toks)):
    (tok, provt) = k.split('_', 1) 
    for (src, tip), cnt in v.items():
        provt = f'{tip}_{provt}'
        if provt in t2set:
            for prov in t2set[provt]:
                if (tok, prov) not in freqs:
                    freqs[(tok, prov)] = {'fn0': 0, 'fn1': 0, 'fn2': 0, 'ln': 0, 'tit': 0, }
                freqs[(tok, prov)][tip] += cnt
        else:
            nema[provt] += 1

freqs2 = {}
for k, v in tqdm(freqs.items(), total=len(freqs)):
    tok, prov = k
    tip = i2t[prov].split('_')[0]
    cnt = v[tip]
    cnt_all = sum(v.values())
    if cnt_all > 0:
        freqs2[(tok, prov)] = round(cnt*100 / cnt_all,2)
    
len(nema), sum(nema.values()), freqs[('prof', 1074)], freqs2[('prof', 1074)]

  0%|          | 0/8313926 [00:00<?, ?it/s]

  0%|          | 0/43177209 [00:00<?, ?it/s]

(5099, 2135573, {'fn0': 0, 'fn1': 0, 'fn2': 0, 'ln': 0, 'tit': 4043}, 100.0)

In [ ]:
# to Oracle
import cx_Oracle
tls = 'db202201301230_high'
connection = cx_Oracle.connect(user='mondo', password=open('orapsw.txt').read(),dsn=tls,encoding="UTF-8")
cur = connection.cursor()
try:
    cur.execute('drop table mondo.wiki_name_parts purge') 
except:
    pass
cur.execute('create table mondo.wiki_name_parts (token varchar2(200), prov number(4,0), ratio number(6, 2))')
#cur.execute('delete  from  mondo.wiki_name_parts')
print('loading ...')
batch = []
br = 0
for t_prov, cnt in tqdm(freqs2.items(), total=len(freqs2)):
    tok, prov = t_prov
    if len(bytes(tok, 'utf-8')) < 190:
        batch.append((tok, prov, cnt))
    if len(batch)>10_048:
        cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :2) ', batch) 
        connection.commit()
        batch = []
    br += 1
cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :2) ', batch) 
connection.commit()
    

loading ...


In [127]:
len(batch)

42563037